# Function Calling in Nebius AI Studio

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nebius/ai-studio-cookbook/blob/main/tool-calling/function_calling_1.ipynb)
[![](https://img.shields.io/badge/Powered%20by-Nebius%20AI-orange?style=flat&labelColor=darkblue&color=green)](https://nebius.com/ai-studio)

## Pre requisites

- Nebius API key.  Sign up for free at [AI Studio](https://studio.nebius.com/)



## 1 - Setup

### 1.1 - If running on Google Colab

Add `NEBIUS_API_KEY` to **Secrets** as follows

![](https://github.com/nebius/ai-studio-cookbook/raw/main/images/google-colab-1.png)


### 1.2 - If running locally

Create an `.env` file with NEBIUS_API_KEY as follows

```text
NEBIUS_API_KEY=your_api_key_goes_here
```



## 2 - Install Dependencies

In [1]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
   print("Running on Colab")
   RUNNING_ON_COLAB = True
else:
   print("NOT running on Colab")
   RUNNING_ON_COLAB = False

NOT running on Colab


In [2]:
#if RUNNING_ON_COLAB:
!pip install -q openai python-dotenv  pydantic

## 3 - Load Configuration


In [3]:
import os

## Recommended way of getting configuration
if RUNNING_ON_COLAB:
   from google.colab import userdata
   NEBIUS_API_KEY = userdata.get('NEBIUS_API_KEY')
else:
   from dotenv import load_dotenv
   load_dotenv()
   NEBIUS_API_KEY = os.getenv('NEBIUS_API_KEY')


## quick hack (not recommended) - you can hardcode the config key here
# NEBIUS_API_KEY = "your_key_here"

if NEBIUS_API_KEY:
  print ('✅ NEBIUS_API_KEY found')
  os.environ['NEBIUS_API_KEY'] = NEBIUS_API_KEY
else:
  raise RuntimeError ('❌ NEBIUS_API_KEY NOT found')

✅ NEBIUS_API_KEY found


## 4 - Pick a Model

We will pick a model that supports function calling.

1. Go to **models** tab in [studio.nebius.com](https://studio.nebius.com/)
2. Select **text to text** models
3. Select **function calling** filter
4. Copy the model name.  For example **`Qwen/Qwen3-30B-A3B`**

See screenshot here:

![](https://github.com/nebius/ai-studio-cookbook/raw/main/tool-calling/function-calling-1.png)

**Recomended models:**

- Qwen3 family
   - Qwen/Qwen3-30B-A3B
   - Qwen/Qwen3-235B-A22B
- Deepseek family
   - deepseek-ai/DeepSeek-R1-0528
- Llama
   - meta-llama/Llama-3.3-70B-Instruct




## 5 - Define Function Call

Here we will use [pydantic](https://docs.pydantic.dev/latest/) to define the schema

In [4]:
from pydantic import BaseModel, Field
from typing import Literal

class GetCurrentWeatherParams(BaseModel):
    city: str = Field(..., description="The city to find the weather for, e.g. 'San Francisco'")
    unit: Literal['celsius', 'fahrenheit'] = Field(..., description="The unit to fetch the temperature in")

# Now, simulate a tool call
## in real example you will use an API to get the actual weather data
def get_current_weather(city: str, unit: str):
    return ("The weather in San Francisco is 72 degrees fahrenheit. "
            "It is sunny, with highs in the 80's.")

tools = [{
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": GetCurrentWeatherParams.model_json_schema()
    }
}]

available_tools = {"get_current_weather": get_current_weather}

## 6 - Tool calling

In [5]:
%%time

import json
import os

from openai import OpenAI

client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=NEBIUS_API_KEY,
)


CPU times: user 490 ms, sys: 65.7 ms, total: 555 ms
Wall time: 612 ms


In [6]:
messages=[
    {
        "role": "user",
        "content": "Can you tell me what the temperature will be in San Francisco?",
    },
  ]

chat_completion = client.chat.completions.create(
  model = "Qwen/Qwen3-30B-A3B",
  messages=messages,
  tools = tools,
  tool_choice = {
        "type": "function",
        "function": {
            "name": "get_current_weather"
        }
    }
)

messages.append({
    "role": "assistant",
    "tool_calls": chat_completion.choices[0].message.tool_calls
})


completion_tool_calls = chat_completion.choices[0].message.tool_calls
for call in completion_tool_calls:
    tool_to_call = available_tools[call.function.name]
    args = json.loads(call.function.arguments)
    result = tool_to_call(**args)
    print(result)
    messages.append({
        "role": "tool",
        "content": result,
        "tool_call_id": call.id,
        "name": call.function.name
    })

The weather in San Francisco is 72 degrees fahrenheit. It is sunny, with highs in the 80's.


In [7]:
messages

[{'role': 'user',
  'content': 'Can you tell me what the temperature will be in San Francisco?'},
 {'role': 'assistant',
  'tool_calls': [ChatCompletionMessageToolCall(id='chatcmpl-tool-0c160206cdea4ca6b10cc37e4b592ad7', function=Function(arguments='{"city": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function')]},
 {'role': 'tool',
  'content': "The weather in San Francisco is 72 degrees fahrenheit. It is sunny, with highs in the 80's.",
  'tool_call_id': 'chatcmpl-tool-0c160206cdea4ca6b10cc37e4b592ad7',
  'name': 'get_current_weather'}]